<a href="https://colab.research.google.com/github/alxmzr/Colab/blob/main/HeatMap_With_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Create Your Own Wall Street HeatMap With Python
# https://medium.com/onepagecode/create-your-own-wall-street-heatmap-with-python-27c0597df2db

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import plotly.express as px
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
columns_to_select = ['Date', 'Symbol', 'Open', 'Close']
sp500_daily_df = pd.read_csv('/gdrive/My Drive/sp500_stocks.csv')[columns_to_select]

In [ ]:
sum_per_day_df = sp500_daily_df.groupby('Date').sum().reset_index()
sum_per_day_df['Net'] = sum_per_day_df['Close'] - sum_per_day_df['Open']
sum_per_day_df.sort_values('Net').head()

,Date,Open,Close,Net
2575,2020-03-20,48491.645942,45439.918517,-3051.727426
3069,2022-03-07,88931.735095,86096.328319,-2835.406776
3120,2022-05-18,82769.513969,80301.121672,-2468.392298
2557,2020-02-25,66819.212826,64406.837873,-2412.374953
3107,2022-04-29,87312.065668,84951.504364,-2360.561304


In [ ]:
worst_day = '2020-03-20'
worst_day_df = sp500_daily_df[sp500_daily_df['Date'] == worst_day]
worst_day_df.head()

,Date,Symbol,Open,Close
2575,2020-03-20,MMM,138.070007,124.889999
5760,2020-03-20,AOS,37.130001,34.750000
8945,2020-03-20,ABT,74.050003,68.000000
12130,2020-03-20,ABBV,73.000000,68.820000
15315,2020-03-20,ABMD,142.130005,130.509995


In [ ]:
columns_to_select = ['Symbol', 'Shortname', 'Sector', 'Weight']
company_details_df = pd.read_csv('/gdrive/My Drive/sp500_companies.csv')[columns_to_select]

worst_day_enriched_df = (worst_day_df.set_index('Symbol')
                         .join(company_details_df.set_index('Symbol'))
                         .reset_index()
                         .dropna())
worst_day_enriched_df.head()

,Symbol,Date,Open,Close,Shortname,Sector,Weight
0,MMM,2020-03-20,138.070007,124.889999,3M Company,Industrials,0.002329
1,AOS,2020-03-20,37.130001,34.750000,A.O. Smith Corporation,Industrials,0.000272
2,ABT,2020-03-20,74.050003,68.000000,Abbott Laboratories,Healthcare,0.005428
3,ABBV,2020-03-20,73.000000,68.820000,AbbVie Inc.,Healthcare,0.007063
4,ABMD,2020-03-20,142.130005,130.509995,"ABIOMED, Inc.",Healthcare,0.000347


In [ ]:
worst_day_enriched_df['PcChangeDay'] = ((worst_day_enriched_df['Close']
                                         - worst_day_enriched_df['Open'])
                                         / worst_day_enriched_df['Open']
                                        * 100
                                       ).round(2)
worst_day_enriched_df.head()

,Symbol,Date,Open,Close,Shortname,Sector,Weight,PcChangeDay
0,MMM,2020-03-20,138.070007,124.889999,3M Company,Industrials,0.002329,-9.55
1,AOS,2020-03-20,37.130001,34.750000,A.O. Smith Corporation,Industrials,0.000272,-6.41
2,ABT,2020-03-20,74.050003,68.000000,Abbott Laboratories,Healthcare,0.005428,-8.17
3,ABBV,2020-03-20,73.000000,68.820000,AbbVie Inc.,Healthcare,0.007063,-5.73
4,ABMD,2020-03-20,142.130005,130.509995,"ABIOMED, Inc.",Healthcare,0.000347,-8.18


In [ ]:
# worst_day_sp500_df = pd.read_csv('/gdrive/My Drive/sp500_companies.csv')
worst_day_sp500_df = worst_day_enriched_df

worst_day_enriched_df.sort_values(by=['Symbol','Sector'], inplace=True)

fig = px.treemap(worst_day_sp500_df,
                 path=[px.Constant('Stocks'), 'Sector',  'Symbol'],
                 values='Weight', color='PcChangeDay',
                 color_continuous_scale=[ 'Red', "#8b0000", 'Green'])

# you can change the index if you have multiple plots
fig.data[0].customdata = worst_day_sp500_df['PcChangeDay']
# We use <br> here to break the line between the symbol (label)
# and the daily change that we passed to the customdata above
fig.data[0].texttemplate = "%{label}<br>%{customdata}"
fig.update_traces(textposition="middle center",
                  selector=dict(type='treemap'))
fig.update_layout(margin = dict(t=30, l=10, r=10, b=10))
# This line removes the the color scale from the chart
fig.update(layout_coloraxis_showscale=False)
fig.show()